In [1]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import random

In [2]:
file_path = './data/'

In [3]:
currency_pair = 'Eur_Usd'
years = '2021-2022'

In [4]:
def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback

def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth

def chop(df, lookback=14):
    atr1 = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

In [5]:
def apply_impact(val, impact):
    return float(val) * float(impact)

In [6]:
currency1, currency2 = currency_pair.split('_')
currency1, currency2 = currency1.upper(), currency2.upper()

news = pd.read_csv(file_path + 'events_test.csv')
news.Date = pd.to_datetime(news.Date, utc=True)
news.drop(news[(news['Impact'] != 'low') & (news['Impact'] != 'med') & (news['Impact'] != 'high')].index, inplace=True)
news.loc[news['Impact'] == 'low', 'Impact'] = 1
news.loc[news['Impact'] == 'med', 'Impact'] = 2
news.loc[news['Impact'] == 'high', 'Impact'] = 3
news['Impact'] = pd.to_numeric(news['Impact'])
news['Actual_Class'] = news.apply(lambda row: apply_impact(row['Actual'], row['Impact']), axis=1)
news['Previous_Class'] = news.apply(lambda row: apply_impact(row['Previous'], row['Impact']), axis=1)
news_base = news.loc[news['Currency_Code'] == currency1]
news_counter = news.loc[news['Currency_Code'] == currency2]
news_base.drop(['Currency_Code', 'Actual', 'Previous', 'Actual_Val', 'Forecast_Val', 'Previous_Val'], axis=1, inplace=True)
news_counter.drop(['Currency_Code', 'Actual', 'Previous', 'Actual_Val', 'Forecast_Val', 'Previous_Val'], axis=1, inplace=True)
by_date1 = news_base.groupby('Date')
impact1, actual1, previous1 = by_date1['Impact'].max().reset_index(), by_date1['Actual_Class'].mean().reset_index(), by_date1['Previous_Class'].mean().reset_index()
news_base = news_base.iloc[0:0]
news_base['Date'], news_base['Impact'], news_base['Actual_Class'], news_base['Previous_Class'] = impact1['Date'], impact1['Impact'], actual1['Actual_Class'], previous1['Previous_Class']
by_date2 = news_counter.groupby('Date')
impact2, actual2, previous2 = by_date2['Impact'].max().reset_index(), by_date2['Actual_Class'].mean().reset_index(), by_date2['Previous_Class'].mean().reset_index()
news_counter = news_counter.iloc[0:0]
news_counter['Date'], news_counter['Impact'], news_counter['Actual_Class'], news_counter['Previous_Class'] = impact2['Date'], impact2['Impact'], actual2['Actual_Class'], previous2['Previous_Class']

/Users/mymac/miniforge3/envs/forex/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-6-d8741bcd9782>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_base['Date'], news_base['Impact'], news_base['Actual_Class'], news_base['Previous_Class'] = impact1['Date'], impact1['Impact'], actual1['Actual_Class'], previous1['Previous_Class']
<ipython-input-6-d8741bcd9782>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [7]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_{years}_test.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.reset_index(drop=True, inplace=True)

df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['chop'] = chop(df)
df['vo'] = vo(df['Volume'])

df = pd.merge(df, news_base, how='left', on='Date')
df = pd.merge(df, news_counter, how='left', on='Date')
df.reset_index(drop=True, inplace=True)
df = df.fillna(method='ffill')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [8]:
df

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,...,atr,adx,chop,vo,Impact_x,Actual_Class_x,Previous_Class_x,Impact_y,Actual_Class_y,Previous_Class_y
0,2022-01-03 14:45:00+00:00,1.13230,1.13232,1.13161,1.13167,1.13246,1.13246,1.13176,1.13181,1.13238,...,0.000574,26.004288,0.287985,0.091579,1.0,0.0,0.0,2.0,-2.0,0.0
1,2022-01-03 14:50:00+00:00,1.13168,1.13171,1.13078,1.13083,1.13183,1.13184,1.13092,1.13097,1.13176,...,0.000592,28.622258,0.232355,0.115926,1.0,0.0,0.0,2.0,-2.0,0.0
2,2022-01-03 14:55:00+00:00,1.13084,1.13117,1.12994,1.13056,1.13098,1.13130,1.13008,1.13070,1.13091,...,0.000648,31.571061,0.212260,0.160239,1.0,0.0,0.0,2.0,-2.0,0.0
3,2022-01-03 15:00:00+00:00,1.13057,1.13078,1.13036,1.13053,1.13072,1.13093,1.13049,1.13067,1.13064,...,0.000639,34.663969,0.207214,0.142164,1.0,0.0,0.0,1.0,-1.0,1.0
4,2022-01-03 15:05:00+00:00,1.13052,1.13081,1.13041,1.13079,1.13066,1.13095,1.13056,1.13093,1.13059,...,0.000629,37.531652,0.201239,0.086798,1.0,0.0,0.0,1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58467,2022-10-14 05:35:00+00:00,0.97806,0.97832,0.97789,0.97824,0.97822,0.97848,0.97805,0.97841,0.97814,...,0.000375,25.914276,0.628341,0.013866,1.0,-1.0,1.0,1.0,0.0,0.0
58468,2022-10-14 05:40:00+00:00,0.97827,0.97832,0.97789,0.97791,0.97845,0.97849,0.97806,0.97811,0.97836,...,0.000367,25.464196,0.620317,0.088017,1.0,-1.0,1.0,1.0,0.0,0.0
58469,2022-10-14 05:45:00+00:00,0.97790,0.97819,0.97776,0.97811,0.97808,0.97836,0.97793,0.97829,0.97799,...,0.000388,25.072120,0.641115,0.053665,1.0,-1.0,1.0,1.0,0.0,0.0
58470,2022-10-14 05:50:00+00:00,0.97812,0.97818,0.97780,0.97808,0.97827,0.97835,0.97795,0.97824,0.97820,...,0.000393,25.044173,0.645969,0.059156,1.0,-1.0,1.0,1.0,0.0,0.0


In [9]:
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [13]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, n_bars, pip_movement, use_pullback, pullback_percentage, use_news, hours, bar_lookback):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    lookback = n_bars + 1 if use_pullback else n_bars
    lookforward = -1 if use_pullback else 0
    pip_movement /= divider
    starting_idx = bar_lookback

    for i in range(starting_idx, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Date']]

        if use_news:
            before = curr_date - timedelta(hours=hours)
            after = curr_date + timedelta(hours=hours)
            impacts = df.loc[(df.Date >= before) & (df.Date <= after)][['Impact_x', 'Impact_y']]
            high_base = impacts['Impact_x'].max() == 3
            high_counter = impacts['Impact_y'].max() == 3

            if high_base or high_counter:
                continue
        
        spread = abs(curr_ask_open - curr_bid_open)

        mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
        mid_highs = list(df.loc[df.index[i - bar_lookback:i], 'Mid_High'])
        mid_lows = list(df.loc[df.index[i - bar_lookback:i], 'Mid_Low'])
        mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])

        buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
        sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

        if use_pullback and buy_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            buy_signal = mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        if use_pullback and sell_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            sell_signal = mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        highest_high, lowest_low = max(mid_highs), min(mid_lows)

        if trade is None:
            if buy_signal:
                open_price = float(curr_ask_open)

                stop_loss = lowest_low
                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

            elif sell_signal:
                open_price = float(curr_bid_open)
                
                stop_loss = highest_high
                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    
            closed_trade = True

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [15]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.0, 1.5]
spread_cutoffs = [0.10]
n_bars_vals = [3]
pip_movement_vals = [20]
use_pullback_vals = [True]
pullback_percentages = [0.35]
use_news_vals = [True, False]
hour_vals = [1, 2, 5, 12, 24]
use_hour_lookback_vals = [12, 24]

all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for n_bars in n_bars_vals:
            for pip_movement in pip_movement_vals:
                for use_pullback in use_pullback_vals:
                    for pullback_percentage in pullback_percentages:
                        for use_news in use_news_vals:
                            for hours in hour_vals:
                                for use_hour_lookback in use_hour_lookback_vals:
                                    all_combos.append((risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pullback_percentage, use_news, hours, use_hour_lookback))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_n_bars_val = None
best_pip_movement_val = None
best_use_pullback_val = None
best_pullback_percentage = None
best_use_news_val = None
best_hour = None
best_use_hour_lookback = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pullback_percentage, use_news, hours, use_hour_lookback in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pullback_percentage, use_news, hours, use_hour_lookback)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'n_bars': n_bars, 'pip_movement': pip_movement, 'use_pullback': use_pullback, 'pullback_percentage': pullback_percentage, 'use_news': use_news, 'hours': hours, 'use_hour_lookback': use_hour_lookback})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_n_bars_val = n_bars
        best_pip_movement_val = pip_movement
        best_use_pullback_val = use_pullback
        best_pullback_percentage = pullback_percentage
        best_use_news_val = use_news
        best_hour = hours
        best_use_hour_lookback = use_hour_lookback

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 40

307.0683700000024
Num buys: 51
Num sells: 36
Num trades: 87
Num wins: 47
Num losses: 40
Win streak: 6
Loss streak: 6
Avg pips risked: 0.004371609195402293
Remaining runs: 39
Best reward so far: 307.0683700000024

42.57045999999941
Num buys: 43
Num sells: 30
Num trades: 73
Num wins: 30
Num losses: 43
Win streak: 3
Loss streak: 9
Avg pips risked: 0.0047063013698630165
Remaining runs: 38
Best reward so far: 307.0683700000024

44.95823999999939
Num buys: 43
Num sells: 30
Num trades: 73
Num wins: 30
Num losses: 43
Win streak: 3
Loss streak: 9
Avg pips risked: 0.0047063013698630165
Remaining runs: 37
Best reward so far: 307.0683700000024

123.86011999999738
Num buys: 12
Num sells: 7
Num trades: 19
Num wins: 11
Num losses: 8
Win streak: 4
Loss streak: 3
Avg pips risked: 0.0039047368421052443
Remaining runs: 36
Best reward so far: 307.0683700000024

-344.3500799999964
Num buys: 46
Num sells: 32
Num trades: 78
Num wins: 36
Num losses: 42
Win streak: 3
Loss streak: 8
Avg pips riske

In [24]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('Best pullback percentage: ' + str(best_pullback_percentage))
print('Best use news val: ' + str(best_use_news_val))
print('Best hours: ' + str(best_hour))
print('Best use hour lookback val: ' + str(best_use_hour_lookback))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1006.4120800000136
Best risk/reward ratio: 1.5
Best spread: 0.1
Best n bars val: 3
Best pip movement val: 20
Best use pullback val: True
Best pullback percentage: 0.35
Best use news val: False
Best hours: 12
Best use hour lookback val: False
-----------------------
Top results:
{'reward': 1006, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pullback_percentage': 0.35, 'use_news': False, 'hours': 12, 'use_hour_lookback': False}
{'reward': 886, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pullback_percentage': 0.5, 'use_news': False, 'hours': 5, 'use_hour_lookback': False}
{'reward': 1005, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pullback_percentage': 0.35, 'use_news': False, 'hours': 24, 'use_hour_lookback': False}
{'reward': 686, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, '

In [12]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('Best pullback percentage: ' + str(best_pullback_percentage))
print('Best use news val: ' + str(best_use_news_val))
print('Best hours: ' + str(best_hour))
print('Best use hour lookback val: ' + str(best_use_hour_lookback))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 411.4812299999988
Best risk/reward ratio: 1.5
Best spread: 0.1
Best n bars val: 3
Best pip movement val: 20
Best use pullback val: True
Best pullback percentage: 0.25
Best use news val: False
Best hours: 12
Best use hour lookback val: True
-----------------------
Top results:
{'reward': 411, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pullback_percentage': 0.25, 'use_news': False, 'hours': 12, 'use_hour_lookback': True}
{'reward': 408, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pullback_percentage': 0.25, 'use_news': False, 'hours': 1, 'use_hour_lookback': True}
{'reward': 330, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pullback_percentage': 0.5, 'use_news': True, 'hours': 5, 'use_hour_lookback': True}
{'reward': 306, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pullback_

In [16]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('Best pullback percentage: ' + str(best_pullback_percentage))
print('Best use news val: ' + str(best_use_news_val))
print('Best hours: ' + str(best_hour))
print('Best use hour lookback val: ' + str(best_use_hour_lookback))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 414.2290400000047
Best risk/reward ratio: 1.5
Best spread: 0.1
Best n bars val: 3
Best pip movement val: 20
Best use pullback val: True
Best pullback percentage: 0.35
Best use news val: True
Best hours: 5
Best use hour lookback val: 24
-----------------------
Top results:
{'reward': 307, 'ratio': 1.0, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pullback_percentage': 0.35, 'use_news': False, 'hours': 24, 'use_hour_lookback': 12}
{'reward': 306, 'ratio': 1.0, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pullback_percentage': 0.35, 'use_news': False, 'hours': 12, 'use_hour_lookback': 12}
{'reward': 414, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pullback_percentage': 0.35, 'use_news': True, 'hours': 5, 'use_hour_lookback': 24}
{'reward': 287, 'ratio': 1.0, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pullback_percenta